# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
import citipy as cpy
from citipy import citipy
import pandas as pd
import numpy as np
import json 
import requests
import matplotlib.pyplot as plt
from api_keys import w_api
from pprint import pprint
import scipy.stats as st

## Generate Cities List

In [ ]:
cities_df=pd.DataFrame({"Longitude":np.random.uniform(-180,180,1400), "Latitude":np.random.uniform(-90,90,1400)})
cities_name=[]
countrycode=[]
for i in range(0,len(cities_df['Latitude'])):
    lat=cities_df.iloc[i]['Latitude']
    long=cities_df.iloc[i]["Longitude"]
    city=citipy.nearest_city(lat,long)
    name=city.city_name
    cities_name.append(name)
    country=city.country_code
    countrycode.append(country)
    
cities_df['City Name']=cities_name
cities_df['Country']=countrycode
cities_df.head(10)

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
cities_df = cities_df.drop_duplicates("City Name", keep='first')
cities_df.reset_index(drop=True, inplace=True)
cities_df

In [ ]:
Temp = []
Humidity = []
Wind_speed = []
Cloudiness = []
Longitude = []
Latitude = []

In [ ]:
url = "http://api.openweathermap.org/data/2.5/weather?"
city = cities_df["City Name"]

for c in city:
    query_url = url + "appid=" + w_api + "&q=" + c.replace(" ","+") + "&units=metric"
    response = requests.get(query_url).json()
    
    
    try:
        Temp.append(response["main"]["temp_max"]) 
        Humidity.append(response["main"]["humidity"]) 
        Wind_speed.append(response["wind"]["speed"]) 
        Cloudiness.append(response["clouds"]["all"])
        Longitude.append(response["coord"]["lon"])
        Latitude.append(response["coord"]["lat"])
        print("Retrieving data for " + response["name"])
    except:
        Longitude.append(None)
        Latitude.append(None)
        Temp.append(None)
        Humidity.append(None)
        Wind_speed.append(None)
        Cloudiness.append(None)
        print("Missing city weather data.......skipping")  


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
#Drop Longitude and Latitude of cities before replacing them with the actual values.
cities_df.drop('Longitude',axis = 1, inplace = True)
cities_df.drop('Latitude',axis = 1, inplace = True)

In [ ]:
cities_df["Longitude"] = Longitude
cities_df["Latitude"] = Latitude
cities_df["Max Temp"] = Temp
cities_df["Humidity"] = Humidity
cities_df["Wind speed"] = Wind_speed
cities_df["Cloudiness"] = Cloudiness


In [ ]:
cities_df.dropna(inplace=True)
cities_df.reset_index(drop=True, inplace=True)
cities_df.to_csv('WeatherPy_Data.csv')

In [ ]:
cities_df["Humidity"] = pd.to_numeric(cities_df["Humidity"], errors='coerce')
cities_df["Max Temp"] = pd.to_numeric(cities_df["Max Temp"], errors='coerce')
cities_df["Wind Speed"] = pd.to_numeric(cities_df["Wind speed"], errors='coerce')
cities_df["Cloudiness"] = pd.to_numeric(cities_df["Cloudiness"], errors='coerce')


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
x_val = cities_df["Latitude"]
y_val = cities_df["Max Temp"]

slope, intercept, regress, p, std_err = st.linregress(x_val,y_val)

regress_values = x_val * slope + intercept

Equation = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

plt.scatter(x_val,y_val)

plt.plot(x_val,regress_values,"g-")

plt.xlabel("Latitude")

plt.ylabel("Max Temp (c)")

plt.title("City Latitude vs. Max Temperature (c)")

plt.annotate(Equation,(-25,0),fontsize=15,color="green")

plt.show()

## Latitude vs. Humidity Plot

In [ ]:
x_val = cities_df["Latitude"]
y_val = cities_df["Humidity"]

slope, intercept, regress, p, std_err = st.linregress(x_val,y_val)

regress_values = x_val * slope + intercept

Equation = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

plt.scatter(x_val,y_val)

#plt.plot(x_val,regress_values,"g-")

plt.xlabel("Latitude")

plt.ylabel("Humidity (%)")

plt.title("City Latitude vs. Humidity (%)")

plt.annotate(Equation,(20,36),fontsize=15,color="green")

plt.show()

## Latitude vs. Cloudiness Plot

In [ ]:
x_val = cities_df["Latitude"]
y_val = cities_df["Cloudiness"]

slope, intercept, regress, p, std_err = st.linregress(x_val,y_val)

regress_values = x_val * slope + intercept

Equation = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

plt.scatter(x_val,y_val)

plt.plot(x_val,regress_values,"g-")

plt.xlabel("Latitude")

plt.ylabel("Cloudiness (%)")

plt.title("City Latitude vs. Cloudiness (%)")

plt.annotate(Equation,(20,36),fontsize=15,color="green")

plt.show()

## Latitude vs. Wind Speed Plot

In [ ]:
x_val = cities_df["Latitude"]
y_val = cities_df["Wind Speed"]

slope, intercept, regress, p, std_err = st.linregress(x_val,y_val)

regress_values = x_val * slope + intercept

Equation = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

plt.scatter(x_val,y_val)

plt.plot(x_val,regress_values,"g-")

plt.xlabel("Latitude")

plt.ylabel("Wind Speed (mph)")

plt.title("City Latitude vs. Wind Speed (mph)")

plt.annotate(Equation,(20,36),fontsize=15,color="green")

plt.show()

## Linear Regression

In [ ]:
North, South = cities_df[(mask:=cities_df['Latitude'] >0 )], cities_df[~mask]

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
x_val = North["Latitude"]
y_val = North["Max Temp"]

slope, intercept, regress, p, std_err = st.linregress(x_val,y_val)

regress_values = x_val * slope + intercept

Equation = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

plt.scatter(x_val,y_val)

plt.plot(x_val,regress_values,"g-")

plt.xlabel("Latitude")

plt.ylabel("Max Temp (c)")

plt.title("City Latitude vs. Max Temp (c)")

plt.annotate(Equation,(10,5),fontsize=15,color="green")

plt.show()

In [ ]:
x_val = South["Latitude"]
y_val = South["Max Temp"]

slope, intercept, regress, p, std_err = st.linregress(x_val,y_val)

regress_values = x_val * slope + intercept

Equation = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

plt.scatter(x_val,y_val)

plt.plot(x_val,regress_values,"g-")

plt.xlabel("Latitude")

plt.ylabel("Max Temp (c)")

plt.title("Southern - City Latitude vs. Max Temp (c)")

plt.annotate(Equation,(20,36),fontsize=15,color="green")

plt.show()

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
North

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
x_val = North["Latitude"]
y_val = North["Humidity"]

slope, intercept, regress, p, std_err = st.linregress(x_val,y_val)

regress_values = x_val * slope + intercept

Equation = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

plt.scatter(x_val,y_val)

plt.plot(x_val,regress_values,"g-")

plt.xlabel("Latitude")

plt.ylabel("Humidity (%)")

plt.title("City Latitude vs. Humidity (%)")

plt.annotate(Equation,(10,40),fontsize=15,color="green")

plt.show()

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
x_val = South["Latitude"]
y_val = South["Humidity"]

slope, intercept, regress, p, std_err = st.linregress(x_val,y_val)

regress_values = x_val * slope + intercept

Equation = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

plt.scatter(x_val,y_val)

plt.plot(x_val,regress_values,"g-")

plt.xlabel("Latitude")

plt.ylabel("Humidity (%)")

plt.title("Southern - City Latitude vs. Humidity (%)")

plt.annotate(Equation,(-80,36),fontsize=15,color="green")

plt.show()

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
x_val = North["Latitude"]
y_val = North["Cloudiness"]

slope, intercept, regress, p, std_err = st.linregress(x_val,y_val)

regress_values = x_val * slope + intercept

Equation = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

plt.scatter(x_val,y_val)

plt.plot(x_val,regress_values,"g-")

plt.xlabel("Latitude")

plt.ylabel("Cloudiness (%)")

plt.title("City Latitude vs. Cloudiness (%)")

plt.annotate(Equation,(20,36),fontsize=15,color="green")

plt.show()

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
x_val = South["Latitude"]
y_val = South["Cloudiness"]

slope, intercept, regress, p, std_err = st.linregress(x_val,y_val)

regress_values = x_val * slope + intercept

Equation = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

plt.scatter(x_val,y_val)

plt.plot(x_val,regress_values,"g-")

plt.xlabel("Latitude")

plt.ylabel("Cloudiness (%)")

plt.title("Southern - City Latitude vs. Cloudiness (%)")

plt.annotate(Equation,(-80,36),fontsize=15,color="green")

plt.show()

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
x_val = North["Latitude"]
y_val = North["Wind Speed"]

slope, intercept, regress, p, std_err = st.linregress(x_val,y_val)

regress_values = x_val * slope + intercept

Equation = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

plt.scatter(x_val,y_val)

plt.plot(x_val,regress_values,"g-")

plt.xlabel("Latitude")

plt.ylabel("Wind Speed (mph)")

plt.title("City Latitude vs. Wind Speed (mph)")

plt.annotate(Equation,(20,10),fontsize=15,color="green")

plt.show()

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
x_val = South["Latitude"]
y_val = South["Wind Speed"]

slope, intercept, regress, p, std_err = st.linregress(x_val,y_val)

regress_values = x_val * slope + intercept

Equation = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

plt.scatter(x_val,y_val)

plt.plot(x_val,regress_values,"g-")

plt.xlabel("Latitude")

plt.ylabel("Wind Speed")

plt.title("Southern - City Latitude vs. Wind Speed")

plt.annotate(Equation,(-80,2),fontsize=15,color="green")

plt.show()